In [9]:
pip install sdkmanager

     ---------------------------------------- 0.0/195.1 kB ? eta -:--:--
     -------------------------------------- 195.1/195.1 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for argcomplete from https://files.pythonhosted.org/packages/c4/08/2a4db06ec3d203124c967fc89295e85a202e5cbbcdc08fd6a64b65217d1e/argcomplete-3.5.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
   ---------------------------------------- 43.6/43.6 kB ? eta 0:00:00
  Created wheel for sdkmanager: filename=sdkmanager-0.6.11-py3-none-any.whl size=45789 sha256=55c1b4692a003a8823015c983bbdba1a6039f410099de371d2c88551d75f018e
  Stored i

In [6]:
import os
import subprocess
import re

# Set project directory
PROJECT_DIR = os.getcwd()


def run_command(command):
    #Runs a command, and return the output from the terminal
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    return result.stdout.strip()

#Was problem with jupyter finding some PATH functions. 
os.environ["ANDROID_HOME"] = "C:\\Users\\User\\AppData\\Local\\Android\\Sdk"
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["ANDROID_HOME"], "platform-tools")
os.environ["PATH"] += os.pathsep + r"C:\Users\User\AppData\Local\Android\sdk\build-tools\35.0.0"


print("ANDROID_HOME is now set to:", os.environ["ANDROID_HOME"])


ANDROID_HOME is now set to: C:\Users\User\AppData\Local\Android\Sdk


In [7]:

def get_gradle_command():
    #Checks if gradle wrapper exists or system gradle is to be used. 
    
    gradlew_path = os.path.join(PROJECT_DIR, "gradlew")
    
    if os.path.exists(gradlew_path):
        print("Gradle Wrapper detected. Using project-specific Gradle.")
        return f'"{gradlew_path}"'
    
    #if systtem gradle
    if run_command("which gradle"):
        print("Using system-installed Gradle.")
        return "gradle"
    
    
    print("Gradle not found! Please install Gradle or add Gradle Wrapper to the project")
    return None

def get_android_sdk():

    #Checks if ANDROID_HOME is set
    android_home = os.environ.get("ANDROID_HOME")
    if not android_home:
        print("ANDROID_HOME is not set! Please configure your Android SDK path.")
        return None
    return android_home



In [8]:
def build_project(gradle_cmd):

    #Builds the projects using Gradle
    
    print("Building the project...") #--stacktrace and --info are optional for debugging
    build_output = run_command(f"{gradle_cmd} assembleDebug --no-daemon --stacktrace --info")
    
    
    if "BUILD SUCCESSFUL" in build_output:
        print("Build successful!")
        return True
    else:
        print("Build failed! Check the logs.")
        #print(build_output.stdout)  # Show full Gradle output
        #print(build_output.stderr)  # Show errors (if any)
        print(build_output)
        return False

def find_apk():
    
    #Finds the debug APK in the project directory
    
    print("Searching for APK file...")
    for root, _, files in os.walk(PROJECT_DIR):
        for file in files:
            if file.endswith(".apk") and "debug" in file:
                apk_path = os.path.join(root, file)
                print(f"APK found: {apk_path}")
                return apk_path
    print("Error: APK not found.")
    return None


In [9]:


def get_device():
    
    #Finds a connected device or emulator
    output = run_command("adb devices")
    
    #Regex to find emulator/device type
    devices = re.findall(r"^(\S+)\s+device$", output, re.MULTILINE)

    if not devices:
        print("No device or emulator detected. Please connect a device or start an emulator.")
        return None
    print(f"Device found: {devices[0]}")
    return devices[0]

def get_package_name(apk_path):
    
    #Extracts the package name from the APK using aapt
    
    command = "aapt dump badging " + apk_path
    output = run_command(command)
    match = re.search(r"package: name='([^']+)'", output)
    if match:
        return match.group(1)
    else:
        print("Package name not found.")
        return None

In [10]:

def install_and_run_apk(device, apk_path):
    
    #Installs and launches APK on the device
    print(f"Installing APK on device: {device}...")
    install_output = run_command(f"adb -s {device} install -r -t {apk_path}")
    
    if "Success" not in install_output:
        print("Installation failed!")
        print(install_output)
        return False

    package_name = get_package_name(apk_path)
    if not package_name:
        print("Failed to extract package name.")
        return False

    print(f"Launching {package_name}...")
    #Laucnhes the program
    run_command(f"adb -s {device} shell monkey -p {package_name} -c android.intent.category.LAUNCHER 1")
    
    print("App launched successfully!")
    return True


def run_monkey_test(device, package_name, event_count=1000):
    
    #Runs the Android Monkey test for stress testing
    print(f"Running Monkey test on {package_name} with {event_count} events...")
    monkey_output = run_command(f"adb -s {device} shell monkey -p {package_name} --throttle 300 -v {event_count}")
    
    print("Monkey Test Completed! Check logs for details.")
    return monkey_output


In [11]:

def main():
    #Runs everything and checks everything
    gradle_cmd = get_gradle_command()
    #print(gradle_cmd)
    if not gradle_cmd:
        return
    
    if not get_android_sdk():
        return
    
    if not build_project(gradle_cmd):
        return
    
    apk_path = find_apk()
    if not apk_path:
        return
    
    device = get_device()
    if not device:
        return
    
    if install_and_run_apk(device, apk_path):
        package_name = get_package_name(apk_path)
        if package_name: 
            log_output = run_monkey_test(device, package_name, event_count=1000)
            print(log_output)  # Show Monkey test output
            #print("hej")
if __name__ == "__main__":
    main()


Gradle Wrapper detected. Using project-specific Gradle.
Building the project...
Build successful!
Searching for APK file...
APK found: C:\Users\User\Documents\Year_5\Master_Thesis\Android_testing_setup\app\build\intermediates\apk\debug\app-debug.apk
Device found: emulator-5554
Installing APK on device: emulator-5554...
Launching com.example.firsttest...
App launched successfully!
Running Monkey test on com.example.firsttest with 1000 events...
Monkey Test Completed! Check logs for details.
bash arg: -p
  bash arg: com.example.firsttest
  bash arg: --throttle
  bash arg: 300
  bash arg: -v
  bash arg: 1000
:Monkey: seed=1739425575085 count=1000
:AllowPackage: com.example.firsttest
:IncludeCategory: android.intent.category.LAUNCHER
:IncludeCategory: android.intent.category.MONKEY
// Event percentages:
//   0: 15.0%
//   1: 10.0%
//   2: 2.0%
//   3: 15.0%
//   4: -0.0%
//   5: -0.0%
//   6: 25.0%
//   7: 15.0%
//   8: 2.0%
//   9: 2.0%
//   10: 1.0%
//   11: 13.0%
:Switch: #Intent;action

In [15]:
!adb devices

List of devices attached
emulator-5554	device

